In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [6]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 5.693197727203369, Val Loss - 2.994965076446533, Val Accuracy - 0.546
Epoch 20, Train Loss - 0.35797885060310364, Val Loss - 2.6528470516204834, Val Accuracy - 0.762
Epoch 40, Train Loss - 0.04570154473185539, Val Loss - 2.88309907913208, Val Accuracy - 0.774
Epoch 60, Train Loss - 0.011027270928025246, Val Loss - 3.362964630126953, Val Accuracy - 0.748
Epoch 80, Train Loss - 0.11694278568029404, Val Loss - 3.8336594104766846, Val Accuracy - 0.734
Epoch 100, Train Loss - 0.003396066837012768, Val Loss - 3.0674242973327637, Val Accuracy - 0.767
Epoch 120, Train Loss - 0.0072468859143555164, Val Loss - 3.5584700107574463, Val Accuracy - 0.761
Epoch 140, Train Loss - 0.0011853190371766686, Val Loss - 3.557541847229004, Val Accuracy - 0.763
Epoch 160, Train Loss - 0.004530248697847128, Val Loss - 3.6915526390075684, Val Accuracy - 0.752
Epoch 180, Train Loss - 0.001738211722113192, Val Loss - 3.661057233810425, Val Accuracy - 0.753
Epoch 200, Train Loss - 0.0036895012

In [7]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.775
Initial Loss: 3.2719228267669678


In [9]:
amts = defaultdict(int)
# Run 10 attacks for perturbation of 0.2
for i in range(10):
    attacker = Metattack(data, device=device)
    attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask, lambda_=0.)
    attacker.reset_parameters(seed=i)
    attacker.reset()
    attacker.attack(0.1)

    for edge in attacker._added_edges.keys():
        amts[edge] += 1


Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

In [10]:
amts

defaultdict(int,
            {(49, 2417): 10,
             (7, 2603): 10,
             (57, 2699): 10,
             (21, 2602): 10,
             (86, 625): 10,
             (13, 2609): 10,
             (28, 68): 10,
             (131, 1668): 10,
             (131, 1612): 10,
             (131, 762): 10,
             (131, 2658): 10,
             (79, 1208): 10,
             (64, 131): 10,
             (11, 1938): 10,
             (131, 2656): 10,
             (131, 1161): 10,
             (66, 2300): 10,
             (131, 569): 10,
             (23, 1684): 10,
             (23, 2697): 10,
             (23, 2147): 10,
             (23, 848): 10,
             (32, 986): 10,
             (23, 1164): 10,
             (23, 311): 10,
             (23, 221): 10,
             (10, 2437): 10,
             (27, 1445): 10,
             (86, 1582): 10,
             (131, 1617): 10,
             (57, 2529): 10,
             (4, 2612): 10,
             (72, 2643): 10,
             (61, 2698): 10,
 

In [11]:
sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=True)
sorted_keys = [key for key, value in sorted_list]

In [12]:
edges_to_add = sorted_keys

In [13]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [20]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.19
budget = int(ptb_rate * initial_edge_count)

In [21]:
print(len(edges_to_add))
print(budget)

502
501


In [22]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [23]:
splits = [0, 0.5]
split_dic = defaultdict(list)

In [24]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.0950
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/501 [00:00<?, ?it/s]

Final Accuracy: 0.7510
Accuracy Drop: 0.0240
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0475
Phase 1: Added 250 edges out of requested 250.


Peturbing graph...:   0%|          | 0/262 [00:00<?, ?it/s]

Final Accuracy: 0.7480
Accuracy Drop: 0.0270


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
